In [ ]:
Author: Spencer Weston

Website: [Crockpot Thoughts](https://crockpotthoughts.wordpress.com/)

Last Update: 08/10/2019

# Guide to NBA Bet
In this notebook, we'll overview the usage of the NBA bet project. First, we'll run the project from the top level. And afterwards, we'll dive into some of the details of the projects implementation. You can work through this notebook in two ways. If you view it on Github, you can download a clone of the project and follow along. Or, if you have jupyter installed, you can run the notebook yourself. However, since it is the NBA offseason, the project will have limited functionality.

## Running the Project
The run directory holds the scripts to run the entire projects. There are two modules: all and daily. Daily calls run.all.run_all() one hour before the next game in the schedule. All contains run_all() which runs the entire project.  

Daily is called from the command line and runs on main. To call it at the command line, enter: ~\NBA_bet>python -m run.daily

To run the project on an ad hoc basis, we use the run_all() function:

In [1]:
from NBApredict.run.all import run_all

ModuleNotFoundError: No module named 'database'

In [2]:
run_all()

c:\users\spencer\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Console output will raise a deprecation warning, an issue I've scheduled to fix. 

It's really that simple to run the project. If you've got this far, you've run the whole project. Now let's look at what's going on under the hood.

## Scraping Data
We scrape data from two sources: Basketballreference.com for basketball statistics and Bovada.com for betting lines. We're concerned with three types of data:
1. Team Stats - These statistics are our explanatory variables. They describe Dean Oliver's four factors as described in the [third post](https://crockpotthoughts.wordpress.com/2019/08/05/predicting-nba-games-part-3-the-model/) in my series overviewing the project. 
2. The Schedule - We need the schedule to know when games are played.
3. Betting Lines - Lines to compare our predictions to.

All the data scrapers are held in the "scrapers" folder. To scrape all data, scrapers.scraper.scrape_all() will scrape and store all data in a local SQLite database. You can find the database in the "outputs" folder. When the project is run, it will create this folder for you. The scrape_all function takes three arguments: a database, session, and the league year. Let's set these arguments up:

In [3]:
from scrapers import scraper
from database.database import Database
from sqlalchemy.orm import Session

In [4]:
db = Database()
session = Session(bind=db.engine)
league_year = 2019

On initialization, the Database class will intialize within the working directory. Alternatively, init accepts a file path to the desired database location. We then initialize the Session with the engine, part of the sqlalchemy package, from db. Now, we can pass these arguments to the scrape_all function.

In [5]:
scraper.scrape_all(database=db, session=session, league_year=league_year)

Now, check the outputs folder. You'll find nba_db.db within. It can be viewed with your database software of choice, or you can interact with it through the instantiated Database class. We can see our tables through the db.get_tables() function. Note: If you didn't call run_all() earlier in the notebook, you will not have the 'predictions_2019' table. 

In [6]:
db.get_tables().keys()

dict_keys(['misc_stats_2019', 'odds_2019', 'sched_2019', 'predictions_2019'])

If you'd like, you can only scrape one type of data at a time. Just call the individual scraper's scrape() function.

In [7]:
from scrapers import line_scraper, team_scraper, season_scraper

team_scraper.scrape(db, league_year)
line_scraper.scrape(db, session, league_year)
season_scraper.scrape(db, session, league_year)

True

## Generating Predictions
Now, let's look at predicting games. We can predict all games we have betting lines for, all games on a specific day, or individual games. Predicting all games will store all results in the database while games on a specific day or individual games will return a dictionary with results. This portion of the project suffers from entangled function because too many specifications and data operations are managed within functions. However, predict_all() can be run with a simple call:

In [8]:
from predict import predict

predict.predict_all(db, session, league_year)

Uhh IDK placeholder

In [10]:
from datetime import datetime

date = datetime(2019, 3, 26)
predict.predict_games_on_date(db, session, league_year, date, console_out=True)

NameError: name 'year' is not defined